In [24]:
from src.data import data, analysis


DATA_DIR = "data"
df = data.load_data(DATA_DIR)

In [29]:
# Take only a few liens for the development
df = df.iloc[:100]

In [28]:
from src.data.analysis import get_img_name

def get_imgs_path(productids: list[int], imageids: list[int]):
    if(len(productids) != len(imageids)):
        raise ValueError("productids and imageids should be the same size") 
    
    return [get_img_name(productid, imageid) for productid, imageid in zip(productids, imageids)]

filenames = get_imgs_path(df.loc[:, "productid"], df.loc[:, "imageid"])

In [42]:
import os
from PIL import Image, ImageOps
import numpy as np

imput_img_dir = "data/images/image_train/"
output_img_dir = "data/images/cropped/"
image_size = (250, 250)

if not os.path.exists(output_img_dir):
    os.makedirs(output_img_dir)

for filename in filenames:
    
    img_array = np.asarray(Image.open(imput_img_dir + filename))
    top_line = -1
    right_line = -1
    bottom_line = -1
    left_line = -1

    i = 1
    can_continue = True
    while can_continue:
        if top_line == -1 and img_array[:i].mean() != 255:
            top_line = i
        if bottom_line == -1 and img_array[-i:].mean() != 255:
            bottom_line = i
        if left_line == -1 and img_array[:, :i].mean() != 255:
            left_line = i
        if right_line == -1 and img_array[:, -i:].mean() != 255:
            right_line = i

        if (top_line != -1 and bottom_line != -1
            and left_line != -1 and right_line != -1):
            can_continue = False
        
        i += 1

    new_img_array = img_array[top_line:-bottom_line, 
                              left_line:-right_line,
                              :]
    new_img = Image.fromarray(new_img_array)
    new_width = new_img.width
    new_height = new_img.height

    ratio = new_width - new_height
    padding_value = np.abs(ratio) // 2
    padding = ()
    if ratio > 0:
        padding = (0, padding_value, 0, padding_value)
    else:
        padding = (padding_value, 0, padding_value, 0)
        
    new_img = ImageOps.expand(new_img, padding, (255, 255, 255))
    new_img = new_img.resize(image_size)
    new_img.save(output_img_dir + filename)
